In [6]:
import polars as pl
import pandas as pd
import geopandas as gpd
import datetime

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from src.procedures import fetch_last_data, generate_density_map
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [24]:
# Retrieves the most recently updated data from the database
value = 7

CONNECTION_URI = config.get("CONNECTION_URI")
query = f"""
    SELECT latitude, longitude, acq_date, acq_time, confidence, frp, brightness, second_adm, first_adm
    FROM processed_viirs 
    WHERE acq_date > CURRENT_DATE - INTERVAL '{value} day'"""

processed_viirs = fetch_last_data(query=query, uri_connection=CONNECTION_URI)
df_viirs = processed_viirs.to_pandas()

df_viirs = df_viirs.rename(columns={"frp":"Fire Radiative Power", "second_adm": "Kabupaten/Kota", "first_adm":"Provinsi",
                                  "acq_date":"Tanggal", "confidence":"Confidence", "brightness":"Brightness"})

df_viirs

,latitude,longitude,Tanggal,acq_time,Confidence,Fire Radiative Power,Brightness,Kabupaten/Kota,Provinsi
0,1.09626,101.001694,2023-06-19,1913,Nominal,1.29,304.029999,Siak,Riau
1,-1.24629,116.813202,2023-06-28,1804,Nominal,4.93,328.649994,None,None
2,0.08398,117.484352,2023-07-01,1707,Nominal,1.86,304.059998,None,None
3,0.84778,101.391228,2023-07-16,1906,Nominal,0.71,303.269989,Siak,Riau
4,-6.49370,139.341782,2023-07-17,423,Nominal,6.98,339.970001,Mappi,Papua
...,...,...,...,...,...,...,...,...,...
114519,5.05186,96.616478,2023-09-12,640,Nominal,7.07,337.929993,Bireuen,Aceh
114520,5.27238,95.432167,2023-09-12,640,Nominal,2.72,326.170013,Aceh Besar,Aceh
114521,5.27246,95.458138,2023-09-12,640,Nominal,1.35,327.459991,Aceh Besar,Aceh
114522,5.27366,95.458389,2023-09-12,640,Nominal,3.79,330.480011,Aceh Besar,Aceh


In [ ]:
# df_viirs["Tanggal"] = df_viirs["Tanggal"].astype(str)

# hover_dict = {"latitude":False, "longitude":False, "Tanggal":True, "acq_time":False, 
#               "Confidence":True,"Fire Radiative Power":True, "Kabupaten/Kota":True, 
#               "Provinsi":False, "Brightness":True}


# map_fig = px.density_mapbox(df_viirs, lat="latitude", lon="longitude", z="Brightness",
#                              radius=1, hover_name="Provinsi",
#                              hover_data=hover_dict,
#                             center=dict(lat=-2.5, lon=118), zoom=3.7, color_continuous_scale="matter_r", 
#                             mapbox_style="open-street-map", template="plotly_white"
#                             )

# map_fig.update_layout(autosize=False,width=1200,height=400)
# map_fig.update_layout(margin={"r":1,"t":1,"l":1,"b":1})
# # map_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
# map_fig.update_coloraxes(showscale=True, colorbar=dict(len=0.75, title="Brightness", thickness=15))

# map_fig


In [22]:
def create_line_chart(data: json):


    line_df = df[df["Provinsi"]==province]

    # Upsample to daily frequency and count the number of fires in each day
    line_df = line_df.resample('D', on='Tanggal')['Fire Radiative Power'].count()

    fig = px.line(line_df, x=line_df.index, y=line_df.values,
            labels={"y":"<b>Jumlah Titik Api</b>", "Tanggal":""},template="plotly_white")

    fig.update_layout(autosize=True,width=700,height=400)
#     fig.update_layout(margin={"r":0,"t":1,"l":0,"b":0})
    fig.update_layout(title="Jumlah Titik Api Harian yang Terdeteksi", title_font_size=16)
    fig.update_traces(line_color='indianred')
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
    fig.update_yaxes(title_font=dict(size=12))
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)

    return fig

In [23]:
line_2022 = create_line_chart(df_viirs, "Jawa Timur")
line_2022

c:\Users\PF2L6BL6\Documents\alvin\personal-works\data-engineering-projects\integrated-active-fire-and-polution-dashboard\.venv-pm25\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
# WHERE acq_date > CURRENT_DATE

In [7]:
CONNECTION_URI = config.get("CONNECTION_URI")
query = f"""
    SELECT *
    FROM articles
    WHERE published_date = CURRENT_DATE"""

articles = fetch_last_data(query=query, uri_connection=CONNECTION_URI)
articles = articles.to_pandas()

In [8]:
articles

,id,keywords,title,article_text,url,image,publisher,published_time,published_date
0,833,kebakaran hutan,Alasan Kebakaran Hutan Sulit Untuk Dipadamkan ...,Kebakaran hutan sering terjadi saat musim kema...,https://news.google.com/rss/articles/CBMidGh0d...,https://assets-a1.kompasiana.com/images/avatar...,Kompasiana.com,2023-09-16 02:48:29,2023-09-16
1,834,polusi udara,"Jaga Kesehatan Keluarga, Hindari Bahaya Polusi...",Krjogja.com - Jakarta - Polusi udara di perkot...,https://news.google.com/rss/articles/CBMifGh0d...,https://static.promediateknologi.id/crop/0x0:0...,Krjogja,2023-09-16 02:10:00,2023-09-16
2,824,kebakaran hutan,"Imbas Kebakaran Hutan dan Lahan di Ketapang, P...","Sabtu, 16 September 2023 | 13:40 WIBKETAPANG, ...",https://news.google.com/rss/articles/CBMiZGh0d...,https://media-origin.kompas.tv/lestari/desktop...,Kompas TV,2023-09-16 06:40:00,2023-09-16
3,825,polusi udara,Memperingati Hari Ozon Internasional: Penyadar...,Sama seperti yang diungkapkan oleh Soemarwoto ...,https://news.google.com/rss/articles/CBMieGh0d...,https://blue.kumparan.com/image/upload/fl_prog...,kumparan.com,2023-09-16 06:32:08,2023-09-16
4,826,polusi udara,5 Tips untuk Menghadapi Kualitas Udara yang Bu...,"Fimela.com, Jakarta Akhir-akhir ini, polusi ud...",https://news.google.com/rss/articles/CBMiX2h0d...,https://cdn1-production-images-kly.akamaized.n...,Fimela.com,2023-09-16 06:00:00,2023-09-16
5,835,polusi udara,Kemelut Polusi Udara Jakarta: Inkonsistensi Da...,A+ A-Belakangan ini masyarakat Indonesia tenga...,https://news.google.com/rss/articles/CBMidWh0d...,https://rm.id/images/logo-pandi.png,RM.ID,2023-09-16 01:38:00,2023-09-16
6,836,kebakaran hutan,Ekonomi Hijau dan Kebakaran Hutan | BALIPOST.c...,Oleh I Wayan RamanthaEkonomi hijau atau green ...,https://news.google.com/rss/articles/CBMiVmh0d...,https://www.balipost.com/wp-content/uploads/20...,BALIPOST.com,2023-09-16 00:51:24,2023-09-16
7,827,polusi udara,Mobilitas Warga Jangan Jadi Kambing Hitam Polu...,"Liputan6.com, Jakarta Peneliti Institute for D...",https://news.google.com/rss/articles/CBMieWh0d...,https://cdn-production-assets-kly.akamaized.ne...,Liputan6.com,2023-09-16 05:30:49,2023-09-16
8,828,kebakaran hutan,Foto: Kebakaran Hutan dan Lahan Gunung Bromo B...,Balai Besar Taman Nasional Bromo Tengger Semer...,https://news.google.com/rss/articles/CBMibWh0d...,https://blue.kumparan.com/image/upload/fl_prog...,kumparan.com,2023-09-16 05:14:36,2023-09-16
9,829,kebakaran hutan,Proses Pemadaman Kebakaran Hutan dan Lahan di ...,"Sabtu, 16 September 2023 | 11:47 WIBBANJARBARU...",https://news.google.com/rss/articles/CBMidmh0d...,https://media-origin.kompas.tv/lestari/desktop...,Kompas TV,2023-09-16 04:47:00,2023-09-16


In [10]:
image = articles["image"][2]
title = articles["title"][2]
url = articles["url"][2]
publish = articles["published_time"][2]
image

'https://media-origin.kompas.tv/lestari/desktop/img/lestari_biruv2.png'

In [34]:
card = dbc.Card(
    [
        dbc.CardImg(src=image, top=True),
        dbc.CardBody(
            [
                # html.H4(f"{title}", className="card-title"),
                html.P(
                    f"{title}",
                    className="card-text",
                ),
                dbc.Button("Go somewhere", color="primary", href=f"{url}"),
            ]
        ),
    ],
    style={"width": "18rem"},
)

card

Card(children=[CardImg(src='https://akcdn.detik.net.id/community/media/visual/2023/09/05/nikita-willy-1_169.png?w=200&q=90', top=True), CardBody([P(children='Bupati Giri Sebut Kebakaran Hutan di Ponorogo Ada Unsur Kesengajaan - detikJatim', className='card-text'), Button(children='Go somewhere', color='primary')])], style={'width': '18rem'})

In [13]:
card = dbc.Card(
    [
        dbc.Row(
            [
                dbc.Col(
                    dbc.CardImg(
                        src=image,
                        className="img-fluid rounded-start",
                    ),
                    className="col-md-4",
                ),
                dbc.Col(
                    dbc.CardBody(
                        [
                            html.H4(f"{title}", className="card-title"),
                            # html.P(
                            #     "This is a wider card with supporting text "
                            #     "below as a natural lead-in to additional "
                            #     "content. This content is a bit longer.",
                            #     className="card-text",
                            # ),
                            html.Small(
                                f"Published date: {publish}",
                                className="card-text text-muted",
                            ),
                        ]
                    ),
                    className="col-md-8",
                ),
            ],
            className="g-0 d-flex align-items-center",
        )
    ],
    className="mb-3",
    style={"maxWidth": "540px"},
    color="warning",
    outline=True
)

In [12]:
import datetime
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

from src.procedures import generate_density_map, fetch_last_data, generate_line_chart, generate_top_prov, generate_top_kabkot

from dotenv import dotenv_values

config = dotenv_values("../.env")
CONNECTION_URI = config.get("CONNECTION_URI")

In [10]:
CONNECTION_URI = config.get("CONNECTION_URI")
query = """
    WITH RANKED_DATA AS (
    SELECT 
        address, city, province, air_quality_index, category, updated_at, 
        ROW_NUMBER() OVER (PARTITION BY address, city, province ORDER BY updated_at DESC) AS rn
    FROM 
        air_quality_idn
    WHERE 
        DATE(fetched_date) = CURRENT_DATE
    )
    SELECT 
        *
    FROM 
        RANKED_DATA
    WHERE 
        rn = 1
    ORDER BY 
        air_quality_index DESC
"""

air_quality = fetch_last_data(query=query, uri_connection=CONNECTION_URI)
air_quality = air_quality.to_pandas()

air_quality["province"] = air_quality["province"].replace({
    "Dki Jakarta":"DKI Jakarta", "Ntt":"NTT", "Ntb":"NTB", "Nad":"Aceh"
})

air_quality

,address,city,province,air_quality_index,category,updated_at,rn
0,"Simpang Icon City, Kota Palembang",Palembang,Sumatera Selatan,149,Tidak Sehat,2023-09-17 18:00:00,1
1,"PM29+F87, Jl. Boulevard BSD Tim., Lengkong Gud...",Tangerang Selatan,Banten,118,Tidak Sehat,2023-09-17 18:00:00,1
2,Kantor Bupati Lama OI,Ogan Ilir,Sumatera Selatan,115,Tidak Sehat,2023-09-17 18:00:00,1
3,"Bantar Gebang [MX2V+2C9, RT.001/RW.005, Ciketi...",Bekasi,Jawa Barat,113,Tidak Sehat,2023-09-17 18:00:00,1
4,"4, Jl. Monumen Pancasila Sakti No.1, RT.4/RW.9...",Jakarta,DKI Jakarta,109,Tidak Sehat,2023-09-17 13:00:00,1
...,...,...,...,...,...,...,...
60,"Wahno, Abepura, Kota Jayapura, Papua 99225, Ko...",Jayapura,Papua,28,Baik,2023-09-14 15:00:00,1
61,"Jl. Jenderal Sudirman No.57, RW.11, Klandasan ...",Balikpapan,Kalimantan Timur,27,Baik,2023-09-17 18:00:00,1
62,"Surabaya, Balongsari, Tandes, Kota Surabaya, J...",Surabaya,Jawa Timur,26,Baik,2023-09-17 18:00:00,1
63,"Ktr. Kecamatan Wanea, Jl. 14 Februari No.72, T...",Manado,Sulawesi Utara,25,Baik,2023-09-17 18:00:00,1


In [11]:
air_quality.province.unique()

array(['Sumatera Selatan', 'Banten', 'Jawa Barat', 'DKI Jakarta', 'Jambi',
       'Sulawesi Tengah', 'Lampung', 'Riau', 'Kalimantan Selatan',
       'Maluku Utara', 'Kalimantan Tengah', 'Kalimantan Timur',
       'Kalimantan Barat', 'Jawa Tengah', 'Sumatera Barat', 'Yogyakarta',
       'Bangka Belitung', 'Nusa Tenggara Barat', 'Sulawesi Selatan',
       'Kepulauan Riau', 'Aceh', 'Sulawesi Tenggara', 'Gorontalo',
       'Kalimantan_utara', 'Bali', 'Jawa Timur', 'Sulawesi Barat',
       'Papua', 'Sulawesi Utara', 'NTT'], dtype=object)

In [13]:
# Retrieves the most recently updated data from the database
value = 7

CONNECTION_URI = config.get("CONNECTION_URI")
query = f"""
    SELECT latitude, longitude, acq_date, acq_time, confidence, frp, brightness, second_adm, first_adm
    FROM processed_viirs 
    WHERE acq_date > CURRENT_DATE - INTERVAL '{value} day'"""

processed_viirs = fetch_last_data(query=query, uri_connection=CONNECTION_URI)
df_viirs = processed_viirs.to_pandas()

df_viirs = df_viirs.rename(columns={"frp":"Fire Radiative Power", "second_adm": "Kabupaten/Kota", "first_adm":"Provinsi",
                                  "acq_date":"Tanggal", "confidence":"Confidence", "brightness":"Brightness"})

df_viirs

,latitude,longitude,Tanggal,acq_time,Confidence,Fire Radiative Power,Brightness,Kabupaten/Kota,Provinsi
0,-9.41209,124.352997,2023-09-12,453,Nominal,25.809999,342.510010,None,None
1,-8.49632,140.698029,2023-09-15,357,Low,6.220000,333.269989,Merauke,Papua
2,-8.45801,140.915466,2023-09-15,357,Low,11.390000,347.630005,Merauke,Papua
3,-8.49632,140.698029,2023-09-15,357,Low,6.220000,333.269989,Merauke,Papua
4,-8.59008,140.628204,2023-09-11,332,Nominal,4.980000,329.019989,Merauke,Papua
...,...,...,...,...,...,...,...,...,...
21516,1.05250,102.266479,2023-09-17,644,Nominal,7.360000,340.230011,Kepulauan Meranti,Riau
21517,1.05540,102.262177,2023-09-17,644,Nominal,5.130000,336.179993,Kepulauan Meranti,Riau
21518,1.16556,101.023689,2023-09-17,646,Nominal,3.700000,342.549988,Siak,Riau
21519,1.35034,101.259758,2023-09-17,646,Nominal,8.280000,346.019989,Bengkalis,Riau


In [25]:
grouped = df_viirs.groupby(["Tanggal"]).agg(
    total_fires = ("Fire Radiative Power", "count")).reset_index()

grouped["Tanggal"] = grouped["Tanggal"].dt.strftime('%Y-%m-%d')
grouped

,Tanggal,total_fires
0,2023-09-11,4202
1,2023-09-12,932
2,2023-09-13,3786
3,2023-09-14,1205
4,2023-09-15,5275
5,2023-09-16,5400
6,2023-09-17,721


In [26]:
grouped.to_json("../data/viirs-json-sample.json", date_format='iso', orient='split', index=False)

In [28]:
grouped_prov = df_viirs.groupby(["Provinsi"]).agg(
    total_fires = ("Fire Radiative Power", "count")).reset_index()

# grouped_prov["Tanggal"] = grouped_prov["Tanggal"].dt.strftime('%Y-%m-%d')
grouped_prov

,Provinsi,total_fires
0,Aceh,61
1,Bali,3
2,Banten,143
3,Bengkulu,163
4,Daerah Istimewa Yogyakarta,16
5,Dki Jakarta,6
6,Gorontalo,184
7,Jambi,750
8,Jawa Barat,517
9,Jawa Tengah,228


In [29]:
grouped_prov.to_json("../data/viirs-json-sample-prov.json", date_format='iso', orient='split', index=False)